In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [2]:
df = dd.read_csv('NIS_2012_CoreCSV.csv', dtype=object)

List of columns with mixed types
21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,62,63,64,65,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104

### List of HCUP column names:

In [3]:
unique = set([])

test = df.KEY_NIS.unique().compute()

# for t in df.KEY_NIS.itertuples():
#     print(t)
#     break

In [10]:
num = 0
for item in df.KEY_NIS.compute(): #test:
    print(item)
    num += 1
    if num == 4:
        break

10000011
10000148
10000174
10000218


In [5]:
print(len(set(test)))

7296968


In [ ]:
len(df.DX1.unique())

In [ ]:
len(df[df['DX1']=='90']) ##checks column for specified value

In [ ]:
len(df[df['DX1']=='44389']) ##checks column for peripheral vascular disease

### Number of unique variables by column

In [ ]:
for i in range(15):
    print("PR{}: ".format(i+1), len(df['PR{}'.format(i+1)].unique()))

In [ ]:
for i in range(25):
    print("DX{}: ".format(i+1), len(df['DX{}'.format(i+1)].unique()))

### Procedure Matrix

The following parser takes the Procedure codes and their descriptions as elements in the 'PRs' array.

In [ ]:
PRs = []
with open("Procedures.txt", "r") as f:
    for line in f:
        code, labels = line.split("=")[0][4:-2], line.split("=")[1][2:-2]
        PRs.append((code,labels))
PRs[:10]

In [ ]:
PRdict = dict(PRs) # converts the key value pairs to a dictionary

In [ ]:
PRdict.pop('    ') # remove 'Blank' from dictionary since not an ICD code

In [ ]:
PRdict

In [ ]:
# makes new dictionary replacing the 'blanks' in the keys with zeros
new_PRdict = {}
for key in PRdict:
    new_PRdict[key.replace(" ", "0")] = PRdict[key]

In [ ]:
new_PRdict

In [ ]:
Procedure_mtx = pd.DataFrame(columns = new_PRdict.keys()) #create procedure matrix

In [ ]:
Procedure_mtx.head()

#### Procedure Matrix Populator


In [ ]:
icd9_codes = sorted(list(new_PRdict.keys()))

In [ ]:
len(icd9_codes)

In [6]:
# Generate list of procedure columns
prcols = []
for i in np.arange(0,15,1):
    prcols.append("PR{}".format(i+1))
prcols.append("KEY_NIS")

In [7]:
dfpx = df[prcols] # create dataframe of only procedure columns and 'KEY_NIS', which is a unique VISIT identifier

In [8]:
dfpx.head()

,PR1,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PR10,PR11,PR12,PR13,PR14,PR15,KEY_NIS
0,741,7534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000011
1,9547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000148
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000174
3,3722,8856,8853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000218
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000229


In [ ]:
code_to_index = {}
for i, code in enumerate(icd9_codes):
    code_to_index[code] = i

In [ ]:
# Dynamically create the procedure matrix...computationally costly because of nparray reshaping and manipulaiton
# test = 0
total_mat = None
done = False
# curr_num_to_index = len(icd9_codes)
for row in dfpx.itertuples():
#     print(row)
    np_row = np.zeros((1, len(icd9_codes)))
    for r in row:
        if type(r) == str:
            try:
                np_row[0, code_to_index[r.zfill(4)]] = 1
            except:
#                 print(r.zfill(4))
                # update the dictionary so the indexes don't double
                code_to_index[r.zfill(4)] = len(icd9_codes)
                # append to list
                icd9_codes.append(r.zfill(4))
                # add column to the full matrix
                new_col = np.zeros((total_mat.shape[0], 1))
                total_mat = np.hstack((total_mat, new_col))
#                 print(total_mat.shape)
                # add single item to row
                new_code = np.ones((1, 1))
                np_row = np.hstack((np_row, new_code))
#                 print(np_row.shape)
                done = True
                
    try:
        total_mat = np.vstack((total_mat, np_row))
    except:
        total_mat = np_row
    if total_mat.shape[0] % 100000 == 0:
        print("Step {} Complete!".format(total_mat.shape[0]))

np.save("total_mat", total_mat)

In [ ]:
total_mat.shape

In [ ]:
total_mat[92]

In [ ]:
total_mat[1, num_to_index["9547"]]

In [ ]:
np.sum(total_mat[2])

In [ ]:
np_row = np.zeros((1, 4))

In [ ]:
r = np.random.random((4, 4))

In [ ]:
r

In [ ]:
r[:, 1]

In [ ]:
r[1, :]

In [ ]:
np_row[0, 1] = 1

In [ ]:
np_row

In [ ]:

one = np.ones((1, 2))
two = np.zeros((1, 2))

In [ ]:
one

In [ ]:
stack = np.vstack((one, two))

In [ ]:
stack

In [ ]:
np.hstack()

In [ ]:
# Generate list of procedure columns
prcols = []
for i in np.arange(0,15,1):
    prcols.append("PR{}".format(i+1))

In [ ]:
prcols

In [ ]:
# Create df of only 15 procedure code columns for simplicity
dfpx = df[prcols]

In [ ]:
dfpx.head()

In [ ]:
for row in dfpx.rows():
    print(row)
    break

In [ ]:
for row in dfpx.itertuples():
    print(row[1])
    break

In [ ]:
for row in dfpx.itertuples():
    for r in row: # row is a list of strings
        if r in PRdict.keys():
            print(r, "was found in PRdict.")
    break

### Diagnosis Matrix

The following parser takes the Diagnosis codes and their descriptions as elements in the 'DXcol_heads' array.

In [ ]:
DXcol_heads = []
with open("Diagnosis.txt", "r") as f:
    for line in f:
        code, labels = line.split("=")[0][4:-2], line.split("=")[1][2:-2]
        code, labels = line.split("=")[0][4:-2], line.split("=")[1][2:-2]
        DXcol_heads.append((code,labels))
DXcol_heads[:10]

In [ ]:
DXdict = dict(DXcol_heads)

In [ ]:
DXdict

In [ ]:
Diagnosis_mtx = pd.DataFrame(columns = DXdict.keys()) #create diagnosis matrix

In [ ]:
Diagnosis_mtx.head()

### Attempt at dd.get_dummies

In [ ]:
ddf_known = df.categorize() #change to only categorize DXi's and PRi's columns

In [ ]:
ddf_known.DX10

In [ ]:
Diag_mtx = dd.get_dummies(ddf_known["DX1"]) # get_dummies only works for one column
#error: Diag_mtx = dd.concat([dd.get_dummies(ddf_known[col]) for col in ddf_known], axis=1, keys=ddf_known.columns)

In [ ]:
Diag_mtx.head()

In [ ]:
exdf = pd.DataFrame({'ID': ['Sam', 'Alex', 'Sam'], 'A': ['a','b','c'], 'B': ['b','a','b'], 'C': ['c','c','a']})


In [ ]:
exdf

In [ ]:
pd.get_dummies(exdf, columns = ['A','B','C'], sparse=True) #to illustrate that unique identifiers stay paired with observations

In [ ]:
for i,x in enumerate(exdf.columns):
        print(i,x)